In [ ]:
import sys, os, decimal, json, pathlib, glob, errno
from tqdm import tqdm
import pandas as pd
import seaborn as sns
from PIL import Image
import numpy as np

import matplotlib.pyplot as plt
import mplcyberpunk
plt.style.use("cyberpunk")
plt.rcParams['figure.facecolor'] = '#0d1117'
plt.rcParams['axes.facecolor'] = '#0d1117'
plt.rcParams['savefig.facecolor'] = '#0d1117'

In [ ]:
image_names = []
path = pathlib.Path(r'O:\eve_models\training_data\route_y')
for data in glob.glob(f"{path}\*.png"):
    file = data.split('\\')[-1:][0]
    image_names.append(file)

In [ ]:
image_data = {}
f = open(f"{path}\\boxes.json")
image_data_raw = json.load(f)
f.close()

for data in image_data_raw:
    image_data[data['image_name']] = [data['bbox'][0:1][0], data['bbox'][1:2][0]]
    
image_x = []
image_y = []
for image_name in tqdm(image_names):
    image_x.append(image_data[image_name][0])
    image_y.append(image_data[image_name][1])

In [ ]:
min_x = 100
max_x = 100
min_y = 100
max_y = 90

In [ ]:
def distance(x, y):
    if x >= y:
        result = (x - y) * -1
    else:
        result = y - x
    return result

In [ ]:
a_image_names = image_names.copy()
a_image_x = image_x.copy()
a_image_offset_x = [0] * len(image_x)
a_image_y = image_y.copy()
a_image_offset_y = [0] * len(image_x)
orig_image_names = image_names.copy()

for i, image_name in tqdm(enumerate(image_names)):
    rand_x_ints = list(np.unique(np.random.randint(low=image_x[i] - min_x, high=image_x[i] + max_x, size=10)))
    rand_y_ints = list(np.unique(np.random.randint(low=image_y[i] - min_y, high=image_y[i] + max_y, size=10)))
   
    for j, x in enumerate(rand_x_ints):
        mod_image_name = image_name.replace('.png',f'-{x}-{image_y[i]}.png')
        a_image_names.append(mod_image_name)
        a_image_x.append(x)
        a_image_y.append(image_y[i])
        orig_image_names.append(image_name)
        a_image_offset_x.append(distance(image_x[i], x))
        a_image_offset_y.append(0)
        
    for j, y in enumerate(rand_y_ints):
        mod_image_name = image_name.replace('.png',f'-{image_x[i]}-{y}.png')
        a_image_names.append(mod_image_name)
        a_image_x.append(image_x[i])
        a_image_y.append(y)
        orig_image_names.append(image_name)
        a_image_offset_x.append(0)
        a_image_offset_y.append(distance(image_y[i], y))

In [ ]:
len(orig_image_names)

In [ ]:
data = { 'orig_image_name':orig_image_names, 'image_names':a_image_names, 'x':a_image_x, 'y':a_image_y, 'x_offset':a_image_offset_x, 'y_offset':a_image_offset_y}
df = pd.DataFrame(columns=['orig_image_name','image_names','x', 'y', 'x_offset', 'y_offset'], data=data)
df

In [ ]:
df[df['orig_image_name'] == '001160f8-d4d1-11ed-a6da-2cf05d9fe8eb.png']

In [ ]:
#data = { 'image_names':image_names, 'x':image_x, 'y':image_y}
#df = pd.DataFrame(columns=['image_names','x', 'y'], data=data)
#df

In [ ]:
#sns.histplot(df[['y']], bins=60)

In [ ]:
df['x_bins'] = pd.cut(df['x'], bins=range(0,500,10))
df['y_bins'] = pd.cut(df['y'], bins=range(0,600,10))
df

In [ ]:
df['bin_id_x'] = 0
df['bin_id_y'] = 0
for i in tqdm(df.index):
    df.loc[i, 'bin_id_x'] = df.loc[i, 'x_bins'].left
    df.loc[i, 'bin_id_y'] = df.loc[i, 'y_bins'].left
df['tick'] = 1
df
#df.groupby('bin_id')['tick'].sum().sort_values(ascending=True)

In [ ]:
df['bin_x_bin_y'] = df['bin_id_x'].astype(str) + "_" + df['bin_id_y'].astype(str)
df

In [ ]:
df.groupby(['bin_x_bin_y'])['tick'].sum().sort_values()

In [ ]:
def mkdir_p(path):
    try:
        os.makedirs(path)
    except OSError as exc: # Python >2.5
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else: raise
        
def copy(source, destination):
    with open(source, 'rb') as file:
        myFile = file.read()
    with open(destination, 'wb') as file:
        file.write(myFile)

In [ ]:
black_image = np.array([[[0]*3]*500]*600)
black_image = Image.fromarray(black_image, mode='RGB')

In [ ]:
import os, os.path
path = r'O:\eve_models\training_data\route_y_large_vert_class_v4'
unique_bins = df['bin_x_bin_y'].unique()
for bin_id in unique_bins:
    mkdir_p(f"{path}\\{bin_id}")

In [ ]:
source = r'O:\eve_models\training_data\route_y'
image_preload = {}
for name in image_names:
    image_preload[name] = Image.open(f"{source}\\{name}")

In [ ]:
#source = r'O:\eve_models\training_data\route_y'
#dest = r'O:\eve_models\training_data\route_y_large_vert_class_v4'
#for i in tqdm(df.index):
#    black_image = np.array([[[0]*3]*500]*600)
#    black_image = Image.fromarray(black_image, mode='RGB')
#    #this_source = f"{source}\\{df.loc[i, 'orig_image_name']}"
#    img = image_preload[df.loc[i, 'orig_image_name']]
#    black_image.paste(img, (df.loc[i, 'x_offset'], df.loc[i, 'y_offset']))
#    this_dest = f"{dest}\\{df.loc[i, 'bin_x_bin_y']}\\{df.loc[i, 'image_names']}"
#    black_image.save(this_dest)
    #print(df.loc[i, 'bin_x_bin_y'])
    #break
    #copy(this_source, this_dest)

In [ ]:
source = r'O:\eve_models\training_data\route_y'
dest = r'O:\eve_models\training_data\route_y_large_vert_class_v4'

ex_orig_image_name = df['orig_image_name']
ex_image_names = df['image_names']
ex_x_offset = df['x_offset']
ex_y_offset = df['y_offset']
ex_bin_x_bin_y = df['bin_x_bin_y']

black_image = np.array([[[0]*3]*500]*600)
black_image = Image.fromarray(black_image, mode='RGB')

for i, name in tqdm(enumerate(ex_orig_image_name)):
    img = image_preload[ex_orig_image_name[i]]
    black_image_copy = black_image.copy()
    black_image_copy.paste(img, (ex_x_offset[i], ex_y_offset[i]))
    this_dest = f"{dest}\\{ex_bin_x_bin_y[i]}\\{ex_image_names[i]}".replace(".png",".bmp")
    black_image_copy.save(this_dest)
